<a href="https://colab.research.google.com/github/NathVM/GA/blob/main/Neo4JGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Measuring performance of Graph Analytics Algorithms using Node4j graphs

---



Setup:

---



In [ ]:
!pip install neo4j
!pip install py2neo

Imports:

---



In [15]:
import pandas as pd
from py2neo import Graph, Node, Relationship
from neo4j import GraphDatabase, basic_auth

Neo4j connection:

---



In [21]:
driver = GraphDatabase.driver(
  "bolt://44.203.104.228:7687",
  auth=basic_auth("neo4j", "rollouts-orifices-basin"))

graph = Graph("bolt://44.203.104.228:7687", auth=("neo4j", "rollouts-orifices-basin"))



---

Query Function:

In [24]:
def run_query(query, parameters = {}):
    with driver.session(database="neo4j") as session:
        result = session.run(query,parameters)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())


cypher_query = '''
MATCH (n)
RETURN COUNT(n) AS count
LIMIT $limit
'''
run_query(cypher_query, parameters={"limit":10} )

,count
0,0


Dataset: 

https://networkrepository.com/TWITTER-Real-Graph-Partial.php

Shared in the google drive 

In [18]:
from google.colab import drive
drive.mount('/content/drive')

# Map the shared folder 
# https://drive.google.com/drive/folders/113gZK1io1MZGogAULYoBdrlEUHyJcxRh?usp=sharing 
# to your google drive and modify the file path accordingly
file = "/content/drive/MyDrive/Dataset/share/GA/TWITTER-Real-Graph-Partial.edges"
df = pd.read_csv(file)
df.rename(columns = {'1':'source', '2':'target'}, inplace = True)
print(df.head(5))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   source  target
0       2       1
1       3       4
2       4       3
3       3       2
4       2       3


Create Graph :

In [26]:
df = df.head(100)
# create nodes
nodes = set(df['source'].unique()).union(df['target'].unique())
node_dict = {}
for node_id in nodes:
    node = Node("Node", id=str(node_id))
    graph.create(node)
    node_dict[node_id] = node

# create relationships
for _, row in df.iterrows():
    source_node = node_dict[row['source']]
    target_node = node_dict[row['target']]
    relationship = Relationship(source_node, "CONNECTS_TO", target_node)
    graph.create(relationship)

In [27]:
run_query(cypher_query, parameters={"limit":10} )

,count
0,56
